ソースコード4.55　ライブラリのインポートとデータの取込

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os.path

%matplotlib inline

sns.set()

plt.rcParams['font.family'] = 'Yu Mincho'

# C:\sample\rnn_simulationに読み書き
WORK_DIR = os.path.join('C:\\', 'sample', 'rnn_simulation')

dji = pd.read_csv(
    os.path.join(
        WORK_DIR, 
        'data',
        'DJI.csv'
    ), 
    parse_dates=['Date'],
    engine='python'
)
dji = dji.assign(ログリターン=np.log(dji.loc[:, 'Adj Close']).diff())
dji = dji.iloc[1:]

ソースコード4.56　メタパラメータの定義

In [ ]:
batch_size = 8
lookback_size = 64
num_hidden_neurons = 40
num_hidden_layers = 3
learning_rate = 1e-5

* ソースコード4.57　入力値と出力値の箱を用意（RecurrentSimulationNetクラスから抜粋）
* ソースコード4.58　ネットワークの再起型コネクションのある部分（RecurrentSimulationNetクラスから抜粋）
* ソースコード4.59　ネットワークの出力から分布の定義（RecurrentSimulationNetクラスから抜粋）
* ソースコード4.60　ネットワークの出力と学習ステップを定義（RecurrentSimulationNetクラスから抜粋）
* ソースコード4.61　ステートを考慮した学習メソッド（RecurrentSimulationNetクラスから抜粋）

In [ ]:
class RecurrentSimulationNet:
    def __init__(
        self, sess, batch_size, lookback_size,
        num_hidden_neurons, num_hidden_layers,
        learning_rate,  momentum=0.9, training=False
    ):
# --- ソースコード4.57 ---
        self._inputs = tf.placeholder(
            tf.float32, [batch_size, lookback_size], name='inputs'
        )
        self._labels = tf.placeholder(
            tf.float32, [batch_size, lookback_size], name='labels'
        )
# --- ソースコード4.57 ここまで ---

# --- ソースコード4.58 ---
        self._inputs_normed = tf.layers.batch_normalization(
            tf.expand_dims(self._inputs, axis=-1),
            momentum=momentum, training=training
        )
                
        with tf.variable_scope('recurrent_part'):
            rnn_cell = tf.nn.rnn_cell.MultiRNNCell(
                [
                    tf.nn.rnn_cell.LSTMCell(num_units=num_hidden_neurons)
                    for _ in range(num_hidden_layers)
                ]
            )
            self._initial_state = rnn_cell.zero_state(batch_size, tf.float32)

            self._rnn_outputs, self._final_state = tf.nn.dynamic_rnn(
                rnn_cell, 
                self._inputs_normed, 
                initial_state=self._initial_state
            )
# --- ソースコード4.58 ここまで ---
# --- ソースコード4.59 ---
        with tf.variable_scope('distribution'):
            self._dist_loc = tf.squeeze(
                tf.layers.dense(self._rnn_outputs, 1), axis=-1
            )
            self._dist_scale = tf.squeeze(
                tf.nn.softplus(tf.layers.dense(self._rnn_outputs, 1)),
                axis=-1
            )
            self._distributions = tf.contrib.distributions.StudentT(
                df=3.0, loc=self._dist_loc, scale=self._dist_scale
            )
# --- ソースコード4.59 ここまで ---
# --- ソースコード4.60 ---
        with tf.variable_scope('outputs'):
            self._dist_sample_size = tf.get_variable(
                'sample_size', (), dtype=tf.int32, trainable=False
            )
            self._dist_sample = self._distributions.sample(
                self._dist_sample_size
            )
            self._log_likelihood = tf.reduce_mean(
                tf.log(
                    self._distributions.prob(self._labels)
                )
            )
            
        update_ops = tf.get_collection(
            tf.GraphKeys.UPDATE_OPS
        )
        with tf.control_dependencies(update_ops):
            self._training_step = tf.train.AdamOptimizer(
                learning_rate
            ).minimize(
                -self._log_likelihood
            )
    
        self.ssn_saver = tf.train.Saver()    
        if training:
            sess.run(
                tf.global_variables_initializer()
            )
# --- ソースコード4.60 ここまで ---
# --- ソースコード4.61 ---
    def train_one_step(
        self, sess, prev_samples, target_samples, state=None
    ):
        fetches = [
            self._training_step,
            self._log_likelihood,
            self._final_state
        ]
        feed_dict = {
            self._inputs: prev_samples,
            self._labels: target_samples
        }
        if state is not None:
            feed_dict = {
                **feed_dict,
                self._initial_state: state
            }
        return sess.run(
            fetches=fetches, feed_dict=feed_dict
        )
# --- ソースコード4.61 ここまで ---
    
    def sample(
        self, sess, prev_samples, sample_size=1, state=None
    ):
        fetches = [
            self._dist_sample,
            self._final_state
        ]
        feed_dict = {
            self._inputs: prev_samples,
            self._dist_sample_size: sample_size
        }
        if state is not None:
            feed_dict = {
                **feed_dict,
                self._initial_state: state
            }
        return sess.run(fetches=fetches, feed_dict=feed_dict)    
    
    def save(self, sess, path):
        self.ssn_saver.save(sess, path)
        print(f'model saved in {path}')
        
    def restore(self, sess, path):
        self.ssn_saver.restore(sess, path)


ソースコード4.62　エポックに分割した学習のためのメタパラメータ

In [ ]:
max_epochs = 100
steps_per_epoch = 50

ソースコード4.63　ニューラルネットワークの学習

In [ ]:
likelihoods_over_time = np.empty(max_epochs)
tf.reset_default_graph()

with tf.Graph().as_default() and tf.Session() as session:
    rec_sim_net = RecurrentSimulationNet(
        session, batch_size, lookback_size,
        num_hidden_neurons, num_hidden_layers,
        learning_rate, training=True
    )
    for epoch in range(max_epochs):
        random_row_indices = np.random.choice(
            dji.shape[0] - lookback_size * (steps_per_epoch+1) - 1,
            size=batch_size,
            replace=True
        )
        random_rows = [
            dji.iloc[
                random_row:random_row + lookback_size*(steps_per_epoch+1),
                :
            ]
            for random_row in random_row_indices
        ]
        log_returns_for_this_epoch = np.reshape(
            [row[['ログリターン']].values for row in random_rows],
            (batch_size, lookback_size*(steps_per_epoch + 1))
        )
        state = None
        likelihood_mean_epoch = 0.
        
        for training_step in range(steps_per_epoch):
            _, log_likelihood, state = rec_sim_net.train_one_step(
                session,
                log_returns_for_this_epoch[
                    :,
                    lookback_size*training_step:lookback_size*(training_step + 1)
                ],
                log_returns_for_this_epoch[
                    :,
                    lookback_size*training_step + 1:lookback_size*(training_step + 1) + 1
                ],
                state=state
            )
            likelihood_mean_epoch += log_likelihood
        likelihoods_over_time[epoch] = likelihood_mean_epoch / steps_per_epoch
        likelihood_mean_polling_interval = 0.
        print(
            f'[{(100*epoch) // max_epochs:3}%] '
            f'直近{steps_per_epoch}ステップの平均尤度：'
            f'{likelihoods_over_time[epoch]:.5f}'
        )
    rec_sim_net.save(
        session, os.path.join(
            WORK_DIR,
            'models',
            'recurrent_simulation_net'
        )
    )
likelihoods_over_time_copy = likelihoods_over_time.copy()

ソースコード4.64　対数尤度のグラフ化

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(
    x=list(range(likelihoods_over_time.shape[0])),
    y=likelihoods_over_time,
    ax=ax
)

ソースコード4.65　シミュレーション用のメタパラメータ

In [ ]:
sim_batch_size = 1
sim_lookback_size = 1
sim_warmup_size = 64 # ウォームアップステップの回数
sim_steps = 100 # シミュレーションの回数

# シミュレーション値を保存するための配列を予め用意する
gen_returns = np.empty((sim_batch_size, sim_warmup_size + sim_steps))

# 実際のリターンを保存するための配列を用意する。サイズはgen_return と同じ。
real_returns = np.empty_like(gen_returns)

ソースコード4.66　ログリターンの疑似データ生成

In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default() and tf.Session() as session:
    rec_sim_net = RecurrentSimulationNet(
        session, sim_batch_size,
        sim_lookback_size,
        num_hidden_neurons,
        num_hidden_layers,
        learning_rate,
        training=False # 学習ステップでないので、training はFalse
    )
    
    # 学習したウェイトを適用
    rec_sim_net.restore(
        session, os.path.join(
            WORK_DIR, 'models', 'recurrent_simulation_net'
        )
    )
    
    # ウォームアップ
    state = None  # ステートを初期化
    
    # ウォームアップの開始地点をランダムに設定。
    # ここではバッチサイズが1 なので1 つ返ってくる。
    random_row_indices = np.random.choice(
        dji.shape[0] - sim_warmup_size - 1,
        size=sim_batch_size,
        replace=True
    )
    
    # 実リターンを予め用意した配列に保存
    real_returns[:, :] = np.reshape(
        [
            dji.iloc[
                random_row:random_row + sim_warmup_size + sim_steps, :
            ][['ログリターン']].values
            for random_row in random_row_indices
        ],
        real_returns.shape
    )
    
    # ウォームアップステップ用に実データを保存
    gen_returns[:, :sim_warmup_size] = np.reshape(
        [
            dji.iloc[
                random_row:random_row + sim_warmup_size, :
            ][['ログリターン']].values
            for random_row in random_row_indices
        ],
        (sim_batch_size, sim_warmup_size))
    
    for warmup_step in range(sim_warmup_size):
        samples, state = rec_sim_net.sample(
            session,
            np.reshape( # 実データを渡す
                [
                    dji.iloc[
                        random_row + warmup_step, :]
                    [['ログリターン']].values
                    for random_row in random_row_indices
                ],  
                (sim_batch_size, 1)
            ),
            state=state  # state のみ保存して、sample は保存しない
        )
    samples = np.squeeze(samples, axis=-1)

    # シミュレーション
    for sim_step in range(sim_steps):
        samples, state = rec_sim_net.sample(
            session,
            samples,  # シミュレーション値を渡す
            state=state
        )
        samples = np.squeeze(samples, axis=-1)
        gen_returns[:, sim_step + sim_warmup_size] = samples

ソースコード4.67　ログリターンから指数値の計算

In [ ]:
real_returns_df = pd.DataFrame(
    np.transpose(real_returns), columns=['real_returns']
)
real_returns_df = real_returns_df.assign(
    real_pf=np.exp(real_returns_df.loc[:, 'real_returns']).cumprod()
)

gen_returns_df = pd.DataFrame(
    np.transpose(gen_returns), columns=['gen_returns']
)
gen_returns_df = gen_returns_df.assign(
    gen_pf=np.exp(gen_returns_df.loc[:, 'gen_returns']).cumprod()
)

sim_data = real_returns_df.join(gen_returns_df)
sim_data = sim_data.loc[
    :, ['real_pf', 'gen_pf']
].stack().reset_index()
sim_data.columns = ['t', 'type', 'pf_value']

ソースコード4.68　指数値をグラフ化

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(x='t', y='pf_value', hue='type', data=sim_data, ax=ax)